## Тема 3.2 Выборка данных из одной таблицы.

## Описание исходных таблиц

Supabase: https://supabase.com/

В примерах используются три таблицы, каждая отражает разную сущность:

### 1. **users**  
**user_id** (PK) — уникальный идентификатор пользователя  
**age** — возраст пользователя  
**churn_date** — дата ухода пользователя (если ушёл, иначе NULL)  
**city** — город проживания  
**first_name** — имя  
**last_name** — фамилия  
**reg_date** — дата регистрации  
**tariff** — используемый тариф

### 2. **internet**  
**id** (PK) — уникальный идентификатор интернет-сессии  
**mb_used** — количество мегабайт, потраченных за сессию  
**session_date** — дата использования  
**user_id** — идентификатор пользователя, совершившего сессию (FK на users.user_id)

### 3. **messages**  
**id** (PK) — уникальный идентификатор сообщения  
**message_date** — дата отправки сообщения  
**user_id** — идентификатор пользователя, отправившего сообщение (FK на users.user_id)

---

# 1. Виды JOIN в PostgreSQL + Примеры

**JOIN** — оператор для объединения данных из двух и более таблиц по связующему полю (например, user_id). Основные виды JOIN:

| JOIN         | Описание                                                       |
|--------------|----------------------------------------------------------------|
| INNER JOIN   | Только строки с совпадениями в обеих таблицах                  |
| LEFT JOIN    | Все из левой таблицы + совпадения из правой (если есть)        |
| RIGHT JOIN   | Все из правой таблицы + совпадения из левой (если есть)        |
| FULL JOIN    | Все строки из обеих таблиц                                     |
| CROSS JOIN   | Каждая строка левой комбинируется с каждой строкой правой      |

---

### 1.1 INNER JOIN

**Задача:** Показать только интернет-сессии с существующими пользователями.

```sql
SELECT 
  internet.id,
  internet.session_date,
  users.first_name,
  users.last_name
FROM 
  internet
INNER JOIN users ON internet.user_id = users.user_id;
```
**Результат:** Только те сессии, user_id которых есть в users.

---

### 1.2 LEFT JOIN

**Задача:** Посмотреть всех пользователей и их расход трафика. Если не пользовался интернетом — будет NULL.

```sql
SELECT
  users.user_id,
  users.first_name,
  internet.mb_used
FROM
  users
LEFT JOIN internet ON users.user_id = internet.user_id;
```
**Результат:** Показывает всех пользователей, даже если у них нет интернет-сессий.

---

### 1.3 RIGHT JOIN

**Задача:** Посмотреть все интернет-сессии и имена пользователей (если есть). Незарегистрированные user_id — NULL.

```sql
SELECT
  internet.id,
  internet.user_id,
  users.first_name
FROM
  users
RIGHT JOIN internet ON users.user_id = internet.user_id;
```
**Результат:** Показывает все интернет-сессии, даже если связанных сессий в users нет.

---

### 1.4 FULL JOIN

**Задача:** Показать все пользователи и сессии, даже если нет совпадений.

```sql
SELECT
  users.user_id,
  users.first_name,
  internet.id AS session_id,
  internet.mb_used
FROM
  users
FULL JOIN internet ON users.user_id = internet.user_id;
```
**Результат:** Отображает все данные обеих таблиц; если нет совпадений — NULL.

---

### 1.5 CROSS JOIN

**Задача:** Каждая пара пользователь – интернет-сессия.

```sql
SELECT
  users.first_name,
  internet.mb_used
FROM
  users
CROSS JOIN internet;
```
**Результат:** Все возможные комбинации пользователей и интернет-сессий.

---

# 2. UNION ALL

**UNION ALL** — объединяет результаты двух SELECT-запросов и сохраняет дубликаты (в отличие от UNION).

### Пример 1: Все user_id, связанные к интернету или сообщениям

```sql
SELECT user_id FROM internet
UNION ALL
SELECT user_id FROM messages;
```
**Результат:** user_id встречается сколько раз есть в обеих таблицах.

---

### Пример 2: Список событий пользователей (интернет-сессии и сообщения) — дата и id пользователя

```sql
SELECT user_id, session_date AS event_date FROM internet
UNION ALL
SELECT user_id, message_date AS event_date FROM messages;
```
**Результат:** Хронологический список действий: интернет-сессии и сообщения.

---

# 3. GROUP BY

**GROUP BY** — группирует строки по одному/нескольким столбцам и позволяет использовать агрегатные функции (COUNT, SUM, AVG...).

### Пример 1: Сообщения по пользователям

```sql
SELECT
  user_id,
  COUNT(*) AS messages_count
FROM
  messages
GROUP BY
  user_id;
```
**Результат:** Для каждого пользователя — сколько сообщений он отправил.

---

### Пример 2: Количество пользователей в каждом городе

```sql
SELECT
  city,
  COUNT(*) AS user_count
FROM
  users
GROUP BY
  city;
```
**Результат:** Для каждого города — сколько зарегистрировано пользователей.

---

### Пример 3: Сессии по датам для каждого пользователя

```sql
SELECT
  user_id,
  session_date,
  COUNT(*) AS sessions_per_day
FROM
  internet
GROUP BY
  user_id, session_date;
```
**Результат:** Сколько сессий у каждого пользователя за каждый день.

---

# 4. HAVING

**HAVING** — фильтрует группы после GROUP BY. Для агрегатных условий (например, “где пользователей больше 10”).

### Пример 1: Только города с более чем 5 пользователями

```sql
SELECT
  city,
  COUNT(*) AS users_in_city
FROM
  users
GROUP BY
  city
HAVING
  COUNT(*) > 5;
```
**Результат:** Только те города, где зарегистрировано больше 5 пользователей.

---

### Пример 2: Пользователи, которые отправляли более 10 сообщений

```sql
SELECT
  user_id,
  COUNT(*) AS messages_count
FROM
  messages
GROUP BY
  user_id
HAVING
  COUNT(*) > 10;
```
**Результат:** Выведет только тех пользователей, кто отправил более 10 сообщений.

---

### Пример 3: Пользователи, которые в каком-либо дне отправляли более одного сообщения

```sql
SELECT
  user_id,
  message_date,
  COUNT(*) AS msgs_per_day
FROM
  messages
GROUP BY
  user_id, message_date
HAVING
  COUNT(*) > 1;
```
**Результат:** Будут показаны только те дни/пользователи, где было отправлено больше 1 сообщения.

---

# 5. Комбинированный пример

**Задача:** Для каждого пользователя узнать, сколько у него было интернет-сессий и сколько сообщений.

```sql
SELECT
  u.user_id,
  u.first_name,
  u.last_name,
  COUNT(DISTINCT i.id) AS total_sessions,
  COUNT(DISTINCT m.id) AS total_messages
FROM
  users u
LEFT JOIN internet i ON u.user_id = i.user_id
LEFT JOIN messages m ON u.user_id = m.user_id
GROUP BY
  u.user_id, u.first_name, u.last_name;
```

---

# Краткие пояснения

- **JOIN** связывает таблицы и позволяет вытаскивать сразу связанный набор данных.
    - INNER — только совпадающие
    - LEFT — все из левой + совпадения из правой (или NULL)
    - RIGHT — все из правой + совпадения из левой (или NULL)
    - FULL — всё из обеих
    - CROSS — все возможные пары

- **UNION ALL** — "склеивает" результаты двух запросов, **не** убирая дубликаты.

- **GROUP BY** — группировка с агрегацией (подсчётом/суммированием и т.д.) по какому-то признаку.

- **HAVING** — фильтрация уже после группировки, по агрегатному признаку (например, HAVING COUNT(*) > 3).

---